In [1]:
# import the necessary libraries
import os
from os import listdir
from PIL import Image
from numpy import asarray,expand_dims
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
from keras_facenet import FaceNet
import pickle
import cv2
from mtcnn import MTCNN

In [2]:
# we are going to use harr cacade first 
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
# if harr cascade is unable to detect we will keep mtcnn for that case
# Initialize the MTCNN detector
mtcnn = MTCNN()
# we are going to use Facenet architecture for creating the embeddings from faces
model_face = FaceNet()

In [3]:
folder = 'training_images/'
# intialize empty dictionary in which we will store the embeddings with name of the person
database = {}

# iterate through all the images in the training images folder 
for filename in listdir(folder):
    path = folder + filename
    gbr1 = cv2.imread(folder + filename)
    
    # detect the face in the cropped photo :
    harr = HaarCascade.detectMultiScale(gbr1,1.1,4)
    faces_mtcnn = mtcnn.detect_faces(gbr1)
    # if the face is detected then get the width and height
    if len(harr)>0:
        x1, y1, width, height = harr[0]
    
    # if harr cascade is unable to detect the face use mtcnn 
    elif len(faces_mtcnn) > 0:
        x1, y1, width, height = faces_mtcnn[0]['box']

    # if no face is detected in the image just use the top left 10x10 pixels
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    #convert from bgr to rgb
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)  # Convert from OpenCV to PIL
    # convert image as numpy array 
    gbr_array = asarray(gbr)

    # crop the face , resize it and store in face 
    face = gbr_array[y1:y2, x1:x2]
    face = Image.fromarray(face)
    face = face.resize((160, 160))
    face = asarray(face)

    # resize the cv2 window to fit the face and draw a rectangle around the face 
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)       
    cv2.resizeWindow("output", gbr1.shape[0],gbr1.shape[1])
    cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Show the image with the bounding box
    cv2.imshow("output", gbr1)
    # change the image when a key is pressed
    cv2.waitKey(0)

    # facenet takes as input 4 dimensional array so we expand dimension 
    face = expand_dims(face, axis=0)
    signature = model_face.embeddings(face)

    # store the array in the database
    database[os.path.splitext(filename)[0]] = signature

cv2.destroyAllWindows()

1/1 [==============================] - 0s 78ms/step


In [4]:
# make a file named data_processed.pkl and store the database in it
myfile = open("data_processed.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [5]:
# show the database
database

{'aakash': array([[-4.36768383e-02, -1.59247057e-03, -1.93988793e-02,
         -4.80910093e-02, -4.99496572e-02, -4.55016550e-03,
          4.46145125e-02, -2.09421609e-02,  4.75572981e-02,
          1.10484157e-02,  2.56375894e-02,  9.43905767e-03,
         -1.99417453e-02,  3.16507593e-02,  5.03975200e-03,
         -5.09092724e-03, -4.43572104e-02,  3.65429670e-02,
         -6.35050833e-02,  4.68730107e-02,  3.07432120e-03,
         -4.29537706e-02,  2.71806531e-02,  1.96834635e-02,
         -5.21697327e-02,  3.07505559e-02, -2.40731519e-02,
          2.09937710e-02, -5.16713597e-02, -4.67510819e-02,
          2.89116930e-02,  2.66712811e-02, -1.38781201e-02,
          8.87357667e-02,  2.34451797e-02,  7.13736191e-02,
          3.54978032e-02,  4.78811376e-02, -1.22598615e-02,
          3.86136174e-02,  6.15890548e-02, -7.05464929e-02,
         -8.36644042e-03,  3.20453383e-02, -3.05437371e-02,
         -7.90913552e-02, -3.15288119e-02,  7.43131489e-02,
         -1.89357307e-02, -1.3